In [1]:
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
import time
from io import StringIO
from pandas.api.types import is_numeric_dtype
import requests as req
from bs4 import BeautifulSoup
import json
import unidecode

import sys
import requests as rq
from bs4 import BeautifulSoup as bs
from time import sleep
from time import time
from random import randint
from warnings import warn

In [376]:
op = webdriver.ChromeOptions()

op.add_argument('--headless')

driver = webdriver.Chrome('/Applications/chromedriver', options=op)
driver2 = webdriver.Chrome('/Applications/chromedriver', options=op)

In [3]:
# ONS LATEST INSIGHTS APRIL Wayback machine archive urls
url = 'http://web.archive.org/cdx/search/cdx?url=https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/articles/coronaviruscovid19/latestinsights&collapse=digest&from=20210707&to=20220501&output=json'
# url = 'http://web.archive.org/cdx/search/cdx?url=ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/articles/coronaviruscovid19roundup/2020-03-26&collapse=digest&from=20200101&to=20211231&output=json'

urls = rq.get(url).text
parse_url = json.loads(urls) #parses the JSON from urls.

## Extracts timestamp and original columns from urls and compiles a url list.
url_list = []
for i in range(1,len(parse_url)):
    orig_url = parse_url[i][2]
    tstamp = parse_url[i][1]
    waylink = tstamp+'/'+orig_url
    url_list.append(waylink)
## Compiles final url pattern.
final_urls = []
for url in url_list:
    final_urls.append('https://web.archive.org/web/'+url)

In [377]:
d = {'page': [], 'section': [], 'page': [], 'date': [], 'roundup': [], 'bulletin_url': [], 'bulletin_mps': []}
df = pd.DataFrame(data=d)
df

,page,section,date,roundup,bulletin_url,bulletin_mps


In [237]:
def getbulurl(link, n):
    driver2.get(link)
    
    ps = driver2.find_elements_by_xpath(f"//*[@id='main']/div[2]/div[1]/article/div[{n}]/section//p")
    
    date = [i.text for i in ps if len(i.text) < 18][0]
    reldate = "Released: " + date.rstrip()
    
    print(reldate)
        
    atags = [i for i in driver2.find_elements_by_xpath(f"//*[@id='main']/div[2]/div[1]/article/div[{n}]/section//a[1]")]
    atags.reverse()
    words = ['Back to the top', 'Data']
    for word in words:
        atags = [i for i in atags if (word not in i.text)]
        
    url1 = [i.get_attribute("href") for i in atags][0]
    
    if url1.split("/")[-1] == 'latest':
        print(url1)
        driver2.get(url1)
        urlPrev = driver2.find_element_by_link_text("View previous releases").get_attribute("href")
        
        url = 'null'
        
        for i in range(1, 20):
            driver2.get(urlPrev + "?page=" + str(i) )
            lis = driver2.find_elements_by_xpath('//*[@id="main"]/div[2]/div/div[1]/ul/li')
            listoas = [i.find_element_by_tag_name("a") for i in lis if i.find_element_by_tag_name("p").text == reldate]
            if len(listoas) > 0:
                url = listoas[0].get_attribute("href")
                break
    
        if url == 'null':
                
            urlPrev = 'https://www.ons.gov.uk/economy/economicoutputandproductivity/output/bulletins/economicactivityandsocialchangeintheukrealtimeindicators/previousReleases'

            for i in range(1, 20):
                driver2.get(urlPrev + "?page=" + str(i) )
                lis = driver2.find_elements_by_xpath('//*[@id="main"]/div[2]/div/div[1]/ul/li')
                listoas = [i.find_element_by_tag_name("a") for i in lis if i.find_element_by_tag_name("p").text == reldate]
                if len(listoas) > 0:
                    url = listoas[0].get_attribute("href")
                    break
 
    else:
        url = url1
    
    return url

getbulurl('https://web.archive.org/web/20210126114928/https://www.ons.gov.uk/economy/economicoutputandproductivity/output/articles/ukeconomylatest/2021-01-25', 3)

Released: 26 January 2021
https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/earningsandworkinghours/bulletins/earningsandemploymentfrompayasyouearnrealtimeinformationuk/latest


'https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/earningsandworkinghours/bulletins/earningsandemploymentfrompayasyouearnrealtimeinformationuk/january2021'

In [238]:
def getru(link, df):

    driver.get(link)    
    s = 1
    
    for n in range(3, 16):
        
        for i in range(0, 4):
        
            n1 = n
            n2 = 1+i

            print('n: ', n, 's: ', s, 'n1: ', n1, 'n2: ', n2)

            if driver.find_elements_by_xpath(f"//*[@id='main']/div[2]/div[1]/article/div[{n1}]/section/h2[{n2}]"):

                content = ""

                content = content + driver.find_element_by_xpath(f"//*[@id='main']/div[2]/div[1]/article/div[{n1}]/section/h2[{n2}]").text + "\n"

                ps = driver.find_elements_by_xpath(f"//*[@id='main']/div[2]/div[1]/article/div[{n}]/section//p")

                for i, j in enumerate(ps):
                    if (i != 0) & (j.text != '') & (j.text not in ['', 'Back to the top', 'Analysis | Data']):
                        content = content + j.text + "\n"

                if content not in [i for i in df['roundup']]:
                    df = df.append({'page': link, 'section': s, 'roundup': content, 'bulletin_url': getbulurl(link, n), 'bulletin_mps': 'placeholder'}, ignore_index=True)
    
    return df

In [345]:
print(len(final_urls))
final_urls[164]

301


'https://web.archive.org/web/20220119130545/https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/articles/coronaviruscovid19/latestinsights'

In [365]:
len('83 September 2022')

17

In [372]:
def testrun(link):
    driver.get(link)    
    s = 0
    piece = {}
    links = {}
    dates = {}
    uninc = ['XLSX', 'Download', 'user requested data', 'Explore the latest insights', 'Read the latest insights', 'See the latest insights', 'Explore the latest indicators', 'Read the latest indicators', 'See the latest indicators']
    linkwords = ['Read the bulletin', 'Read the latest', 'Latest bulletin', 'Read ']
    
    for n in range(3, 16):

        if driver.find_elements_by_xpath(f'//*[@id="main"]/div[2]/div[1]/article/div[{n}]/section/*'):
            for el in driver.find_elements_by_xpath(f'//*[@id="main"]/div[2]/div[1]/article/div[{n}]/section/*'):
                
                if (len(el.text) < 18) & (('2021' in el.text)|('2022' in el.text)):
                    dates[s].append(el.text)
                    
                if el.text == 'Also published recently':
                    return [piece, links, dates]
                
                
                linky = False
                if (el.tag_name == 'h2')|(el.tag_name == 'h3'):
#                     print(" *** ")
                    s = s+1
                    piece[s] = []
                    links[s] = []
                    dates[s] = []
                    piece[s].append(el.text)
                if (el.tag_name == 'p') & (el.text not in ['','🠕 Back to the top']):
                    
                    inc = True
                    
                    # Check if contains a link
                    if el.find_elements_by_xpath("a"):
                        for w in uninc:
                            if w in el.find_element_by_xpath("a").text:
                                inc = False

                        for w in linkwords:
                            if w in el.find_element_by_xpath("a").text:
                                linky = True

                        if linky:
                            links[s].append({'text': el.text, 'link': el.get_attribute("href")})

                        elif inc:
                            piece[s].append(el.text)

                            
                    else:
                        piece[s].append(el.text)
                            
                            
                if (el.tag_name == 'ul') & (el.text not in ['','🠕 Back to the top']):
                    for li in el.find_elements_by_xpath('li'):
                        
                        inc = True
                        
                        if li.find_elements_by_xpath("p"):
                            li = li.find_element_by_xpath("p")                        
                        
                        # Check if contains a link
                        if li.find_elements_by_xpath("a"):
                            for w in uninc:
                                if w in li.find_element_by_xpath("a").text:
                                    inc = False

                            for w in linkwords:
                                if w in li.find_element_by_xpath("a").text:
                                    linky = True

                            if linky:
                                links[s].append({'text': li.text, 'link': li.find_element_by_xpath("a").get_attribute("href")})

                            elif inc:
#                                 print('li ', li.find_element_by_xpath("a").text )
                                piece[s].append(" - " + li.text)

                        else:
                            piece[s].append(" - " + li.text)

                
    return [piece, links, dates]

pieces = testrun(final_urls[164])
print(pieces[0][1])
print(pieces[1][1])

['Infections decrease in England, Wales and Scotland', 'Early estimates suggest around 1 in 20 people in England would have tested positive for coronavirus (COVID-19) in the week to 15 January 2022.', 'During this period of high infections, we are publishing headline results of the Coronavirus Infection Survey two days early. These are subject to change as more data comes in but are based on an acceptable number of tests.', 'The percentage of people testing positive for COVID-19 decreased in England, Wales and Scotland, while the trend in Northern Ireland was uncertain in the latest week.', 'COVID-19 infections compatible with the Omicron variant continue to be the most dominant across all UK countries.', 'The estimated number of people living in private households (not in hospitals, care homes and/or other communal establishments) that had COVID-19 in the most recent week was:', ' - England – 2,984,200 people (1 in 20)', ' - Wales – 112,100 people (1 in 25)', ' - Northern Ireland – 10

In [373]:
print(pieces[0][6])
print(pieces[1][6])
print(pieces[2][1])

['More than 96% of the population have COVID-19 antibodies', 'In England, it is estimated that 97.5% of the adult population (95% credible interval: 97.0% to 97.9%) would have tested positive for antibodies against SARS-CoV-2 in the week beginning 20 December 2021.', 'In Wales, an estimated 96.8% of the adult population (95% credible interval: 95.9% to 97.5%) would have tested positive for antibodies against SARS-CoV-2 in the same week.', 'In Northern Ireland, it is estimated that 97.4% of the adult population (95% credible interval: 96.0% to 98.2%) would have tested positive for antibodies against SARS-CoV-2 in the same week.', 'In Scotland, it is estimated that 97.7% of the adult population (95% credible interval: 97.1% to 98.2%) would have tested positive for antibodies against SARS-CoV-2 in the same week.', 'Academic research on antibody thresholds using data from when the Delta variant was the dominant strain indicate that a higher threshold of antibodies is needed to provide prot

In [374]:
def addpiecesandlinks(url, df):
    pieces = testrun(url)
    for k in pieces[0].keys():
        if pieces[0][k] not in [i for i in df['roundup']]:
            df = df.append({'page': url, 'section': k, 'date': pieces[2][k], 'roundup': pieces[0][k], 'bulletin_url': pieces[1][k], 'bulletin_mps': 'placeholder'}, ignore_index=True)
    return df

In [379]:
for i, url in enumerate(final_urls):
    if i>-1:
        print(i, url)
        df = addpiecesandlinks(url, df)

0 https://web.archive.org/web/20210709115515/https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/articles/coronaviruscovid19/latestinsights
1 https://web.archive.org/web/20210709115753/https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/articles/coronaviruscovid19/latestinsights
2 https://web.archive.org/web/20210709175237/https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/articles/coronaviruscovid19/latestinsights
3 https://web.archive.org/web/20210711082001/https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/articles/coronaviruscovid19/latestinsights
4 https://web.archive.org/web/20210712170449/https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/articles/coronaviruscovid19/latestinsights
5 https://web.archive.org/web/20210713150217/https://www.ons.gov.uk/peoplepopulationandcom

In [351]:
df = df.iloc[:-5 , :]


In [498]:
def checklatest(url, date):
    if url == None:
        return "ERR: URL IS NONE"
    bulletinurl = "ERR: NOTHING SELECTED"
    reldate = "Released: " + date.rstrip()
    e = "https:/" + url.split("https:/")[-1]
    e = e.replace("https:/www", "https://www")
    print(e)
    if e.split("/")[-1] == 'latest':

        req = rq.get(e).text
        soup = bs(req,'html.parser')
        prevrelurl = "https://www.ons.gov.uk" + [i for i in soup.find_all("a") if i.text =='View previous releases'][0]['href']

        linkseg = 'null'
        for i in range(1, 20):
            req = rq.get( prevrelurl + "?page=" + str(i) ).text
            soup = bs(req,'html.parser')

            if '404 - ' not in soup.find('title').text:


                lis = soup.find("main").find("div", class_="print-links").find_all('li')

                if len([i.find("a") for i in lis if reldate in i.text]) > 0:
                    linkseg = [i.find("a") for i in lis if reldate in i.text][0]['href']
                    break

        if linkseg == 'null':

            if 'Coronavirus and the social impacts' in soup.find('h1').text:
                prevrelurl = "https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/healthandwellbeing/bulletins/coronavirusandthesocialimpactsongreatbritain/previousReleases"

            elif 'Economic activity and social change' in soup.find('h1').text:
                prevrelurl = 'https://www.ons.gov.uk/economy/economicoutputandproductivity/output/bulletins/economicactivityandsocialchangeintheukrealtimeindicators/previousReleases'

            for i in range(1, 20):
                req = rq.get( prevrelurl + "?page=" + str(i) ).text
                soup = bs(req,'html.parser')

                if '404 - ' not in soup.find('title').text:
                    lis = soup.find("main").find("div", class_="print-links").find_all('li')

                    if len([i.find("a") for i in lis if reldate in i.text]) > 0:
                        linkseg = [i.find("a") for i in lis if reldate in i.text][0]['href']
                        break

        if linkseg == 'null':
            belletinurl = "ERR: NO PREVIOUS RELEASE FOR :: " + e + " DATE :: " + date
        else:
            bulletinurl = "https://www.ons.gov.uk" + linkseg

    elif '/releases/' not in e:
        bulletinurl = e

    else:    
        req = rq.get(e).text
        # parse roundup html using beautifulsoup and store in soup
        soup = bs(req,'html.parser')

        cols = soup.find_all('div', class_="col-wrap")

        pubs = [i for i in cols if 'Publications' in i.text]

        if len(pubs)>0:
            pub = pubs[0]
            pubind = cols.index(pub)
            linksect = cols[pubind+1]
            bulletinurl = "https://www.ons.gov.uk" + linksect.find("a")['href']
        else:
            bulletinurl = "ERR: NO PUBLICATION FOR :: " + e + " DATE :: " + date

    bulletinurl = bulletinurl.replace("https://ons", "https://www.ons")

    return bulletinurl
checklatest(df['bulletin_url'][79][-1]['link'], df['date'][79][0])

https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/bulletins/prevalenceofongoingsymptomsfollowingcoronaviruscovid19infectionintheuk/latest


'ERR: NOTHING SELECTED'

In [463]:
# dfcopy = df.copy()

In [464]:
# df_nourl = df[df.apply(lambda x: x['bulletin_url'] == [], axis=1)]
# df = df[df.apply(lambda x: x['bulletin_url'] != [], axis=1)]

In [499]:
for i in df.index:
    if i > 76:
        print(i)
        df['bulletin_mps'][i] = checklatest(df['bulletin_url'][i][-1]['link'], df['date'][i][0])

77
https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/bulletins/coronaviruscovid19infectionsurveypilot/latest


<ipython-input-499-10ced233ddba>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['bulletin_mps'][i] = checklatest(df['bulletin_url'][i][-1]['link'], df['date'][i][0])


78
https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/bulletins/coronaviruscovid19infectionsurveyantibodyandvaccinationdatafortheuk/latest
79
https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/bulletins/prevalenceofongoingsymptomsfollowingcoronaviruscovid19infectionintheuk/latest
80
https://www.ons.gov.uk/releases/economicactivityandsocialchangeintheukrealtimeindicators16september2021
81
https://www.ons.gov.uk/releases/covid19infectionratesbysociodemographiccharacteristicsengland1september2020to25july2021
82
https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/bulletins/coronaviruscovid19infectionsurveypilot/latest
83
https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/healthandwellbeing/bulletins/coronavirusandthesocialimpactsongreatbritain/latest
84
https://www.ons.gov.uk/releases/economicactivityandsocialchangeintheukrealtimeindicators23s

In [502]:
# df['bulletin_mps'] = 'ph'
# df['final_url'] = df['bulletin_mps']

<ipython-input-502-9c1248ec9919>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['bulletin_mps'] = 'ph'


In [507]:
df_nofinurl = df[df.apply(lambda x: x['final_url'][:3] == 'ERR', axis=1)]
df = df[df.apply(lambda x: x['final_url'][:3] != 'ERR', axis=1)]

In [ ]:
df_nofinurl = df[df.apply(lambda x: x['final_url'][:3] == 'ERR', axis=1)]
df = df[df.apply(lambda x: x['final_url'][:3] != 'ERR', axis=1)]

In [511]:
def getmps(bulletinurl):
    bulletinurl = "http" + bulletinurl.split("http")[-1]
    
    # Open page for roundup
    req = rq.get(bulletinurl).text
    # parse roundup html using beautifulsoup and store in soup
    soup = bs(req,'html.parser')
        
    sects = [sect for sect in soup.find_all('section') if sect.find_all('h2') != []]    
    sect = [sect for sect in sects if 'Main points' in sect.find('h2').text]
    
    if len(sect)>0:
                
        sect = sect[0]

        lis = sect.find('ul')
        if lis != None:
            sibling = lis.next_sibling

            while sibling:
                next_sibling = sibling.next_sibling
                sibling.extract()
                sibling = next_sibling
            
        ps = []
        for i in sect:
            if i.name == 'p':
                ps.append(i.text)

            elif i.name == 'ul':
                for j in i.find_all('li'):
                    ps.append(j.text)            

    else:
        return "NO MAIN POINTS"
        
    mps = [i for i in ps if ('Source: Office for National Statistics' not in i) and ('Find out more' not in i) and ('make the website work as well as possible' not in i) and ('Embed this interactive' not in i) and ('xlsx' not in i) and ('cookies' not in i)]
    mainpoints = ""
    for i in mps: 
        mainpoints = mainpoints + i + '\n' 
    
    return mainpoints

# print(getmps(df_wi_bul['bulletin_url'][0][0]))

In [514]:
for i in df.index:
    if i > 307:
        print(i)
        df['bulletin_mps'][i] = getmps(df['final_url'][i])

308


<ipython-input-514-f760d936ef8e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['bulletin_mps'][i] = getmps(df['final_url'][i])


309
310
311
312
313
314
315
316
317


In [519]:
df

,page,section,date,roundup,bulletin_url,bulletin_mps,final_url
0,https://web.archive.org/web/20210709115515/htt...,1.0,[9 July 2021],"[Infections have increased across the UK, Coro...",[{'text': 'Read the latest Coronavirus (COVID-...,"In England, the percentage of people testing p...",https://www.ons.gov.uk/peoplepopulationandcomm...
1,https://web.archive.org/web/20210709115515/htt...,2.0,[9 July 2021],[Most report more social distancing than they ...,[{'text': 'Latest bulletin on Coronavirus and ...,"This week, over the period 30 June to 4 July 2...",https://www.ons.gov.uk/peoplepopulationandcomm...
2,https://web.archive.org/web/20210709115515/htt...,3.0,[7 July 2021],[COVID-19 antibodies continue to rise in line ...,[{'text': 'Latest bulletin on antibody and vac...,"In England, it is estimated that around 9 in 1...",https://www.ons.gov.uk/peoplepopulationandcomm...
3,https://web.archive.org/web/20210709115515/htt...,4.0,[6 July 2021],[Deaths in England and Wales fall below five-y...,[{'text': 'Latest bulletin on deaths registere...,"There were 8,690 deaths in England and Wales r...",https://www.ons.gov.uk/peoplepopulationandcomm...
4,https://web.archive.org/web/20210709115515/htt...,5.0,[6 July 2021],[North West and London had highest coronavirus...,[{'text': 'Read Deaths registered due to COVID...,Of all deaths registered in 2020 in England an...,https://www.ons.gov.uk/peoplepopulationandcomm...
...,...,...,...,...,...,...,...
313,https://web.archive.org/web/20220426095424/htt...,1.0,[26 April 2022],[10.1% of registered deaths in England and Wal...,[{'text': 'Read the latest bulletin on deaths ...,"In the week ending 15 April 2022 (Week 15), 9,...",https://www.ons.gov.uk/peoplepopulationandcomm...
314,https://web.archive.org/web/20220430123803/htt...,1.0,[29 April 2022],[Three out of four UK countries show a continu...,[{'text': 'Read the latest results from the Co...,In the week ending 23 April 2022:\nThe percent...,https://www.ons.gov.uk/peoplepopulationandcomm...
315,https://web.archive.org/web/20220430123803/htt...,2.0,[29 April 2022],[UK sickness absence rate hit 10-year high in ...,[{'text': 'Read the latest insights on COVID-1...,The sickness absence rate in the UK in 2021 ro...,https://www.ons.gov.uk/employmentandlabourmark...
316,https://web.archive.org/web/20220430123803/htt...,3.0,[28 April 2022],[54% rise in fraud and computer misuse offence...,[{'text': 'Read more in our Crime in England a...,Patterns of crime over the last two years have...,https://www.ons.gov.uk/peoplepopulationandcomm...


In [521]:
df_nomps = df[df['bulletin_mps'] == 'NO MAIN POINTS']
df = df[df['bulletin_mps'] != 'NO MAIN POINTS']

In [523]:
df_nompsPH = df[df['bulletin_mps'] == 'ph']
df = df[df['bulletin_mps'] != 'ph']

In [525]:
df

,page,section,date,roundup,bulletin_url,bulletin_mps,final_url
0,https://web.archive.org/web/20210709115515/htt...,1.0,[9 July 2021],"[Infections have increased across the UK, Coro...",[{'text': 'Read the latest Coronavirus (COVID-...,"In England, the percentage of people testing p...",https://www.ons.gov.uk/peoplepopulationandcomm...
1,https://web.archive.org/web/20210709115515/htt...,2.0,[9 July 2021],[Most report more social distancing than they ...,[{'text': 'Latest bulletin on Coronavirus and ...,"This week, over the period 30 June to 4 July 2...",https://www.ons.gov.uk/peoplepopulationandcomm...
2,https://web.archive.org/web/20210709115515/htt...,3.0,[7 July 2021],[COVID-19 antibodies continue to rise in line ...,[{'text': 'Latest bulletin on antibody and vac...,"In England, it is estimated that around 9 in 1...",https://www.ons.gov.uk/peoplepopulationandcomm...
3,https://web.archive.org/web/20210709115515/htt...,4.0,[6 July 2021],[Deaths in England and Wales fall below five-y...,[{'text': 'Latest bulletin on deaths registere...,"There were 8,690 deaths in England and Wales r...",https://www.ons.gov.uk/peoplepopulationandcomm...
4,https://web.archive.org/web/20210709115515/htt...,5.0,[6 July 2021],[North West and London had highest coronavirus...,[{'text': 'Read Deaths registered due to COVID...,Of all deaths registered in 2020 in England an...,https://www.ons.gov.uk/peoplepopulationandcomm...
...,...,...,...,...,...,...,...
313,https://web.archive.org/web/20220426095424/htt...,1.0,[26 April 2022],[10.1% of registered deaths in England and Wal...,[{'text': 'Read the latest bulletin on deaths ...,"In the week ending 15 April 2022 (Week 15), 9,...",https://www.ons.gov.uk/peoplepopulationandcomm...
314,https://web.archive.org/web/20220430123803/htt...,1.0,[29 April 2022],[Three out of four UK countries show a continu...,[{'text': 'Read the latest results from the Co...,In the week ending 23 April 2022:\nThe percent...,https://www.ons.gov.uk/peoplepopulationandcomm...
315,https://web.archive.org/web/20220430123803/htt...,2.0,[29 April 2022],[UK sickness absence rate hit 10-year high in ...,[{'text': 'Read the latest insights on COVID-1...,The sickness absence rate in the UK in 2021 ro...,https://www.ons.gov.uk/employmentandlabourmark...
316,https://web.archive.org/web/20220430123803/htt...,3.0,[28 April 2022],[54% rise in fraud and computer misuse offence...,[{'text': 'Read more in our Crime in England a...,Patterns of crime over the last two years have...,https://www.ons.gov.uk/peoplepopulationandcomm...


In [526]:
dforig = pd.read_csv('covidrounduporig.csv')

In [527]:
dforig

,page,section,date,roundup,bulletin_url,final_url,bulletin_mps
0,https://web.archive.org/web/20201109120409/htt...,1.0,['6 November 2020'],['Number of COVID-19 infections continues to i...,"[{'text': 'Analysis', 'link': 'https://www.ons...",https://www.ons.gov.uk/peoplepopulationandcomm...,The infection rate has increased in recent wee...
1,https://web.archive.org/web/20201109120409/htt...,2.0,['3 November 2020'],['Deaths involving COVID-19 increased for the ...,"[{'text': 'Analysis', 'link': 'https://www.ons...",https://www.ons.gov.uk/peoplepopulationandcomm...,The number of deaths registered in England and...
2,https://web.archive.org/web/20201109120409/htt...,4.0,['5 November 2020'],"['Fewer adults are travelling to work', 'The p...","[{'text': 'Analysis', 'link': 'https://www.ons...",https://www.ons.gov.uk/peoplepopulationandcomm...,According to the latest Business Impact of Cor...
3,https://web.archive.org/web/20201109120409/htt...,5.0,['9 November 2020'],['Overseas travel in spring 2020 was down 96% ...,"[{'text': 'Analysis', 'link': 'https://www.ons...",https://www.ons.gov.uk/peoplepopulationandcomm...,"Overseas residents made 398,000 visits to the ..."
4,https://web.archive.org/web/20201109120409/htt...,6.0,['3 November 2020'],['Hospitality workers see largest fall in pay'...,"[{'text': 'Analysis', 'link': 'https://www.ons...",https://www.ons.gov.uk/employmentandlabourmark...,Statistics in this bulletin (based on the Annu...
...,...,...,...,...,...,...,...
223,https://web.archive.org/web/20210808215813/htt...,3.0,['6 August 2021'],['A quarter of\u202fadults\u202fself-isolating...,[{'text': 'Read the latest bulletin on coronav...,https://www.ons.gov.uk/peoplepopulationandcomm...,"This week, over the period 28 July to 1 August..."
224,https://web.archive.org/web/20210808215813/htt...,4.0,['5 August 2021'],['1 in 68 people in the UK experiencing self-r...,[{'text': 'Read the latest bulletin on the pre...,http://ons.gov.uk/peoplepopulationandcommunity...,"An estimated 945,000 people living in private ..."
225,https://web.archive.org/web/20210808215813/htt...,5.0,['5 August 2021'],['Average UK daily flights up 117% since May 2...,[{'text': 'Read the latest bulletin on economi...,https://www.ons.gov.uk/economy/economicoutputa...,"In the week to 1 August 2021, the seven-day av..."
226,https://web.archive.org/web/20210808215813/htt...,6.0,['4 August 2021'],['9 out of 10 adults tested positive for COVID...,[{'text': 'Read more in the Coronavirus (COVID...,https://www.ons.gov.uk/peoplepopulationandcomm...,"In England, it is estimated that over 9 in 10 ..."


In [528]:
cols = df.columns.tolist()


In [529]:
cols

['page',
 'section',
 'date',
 'roundup',
 'bulletin_url',
 'bulletin_mps',
 'final_url']

In [531]:
cols.insert(6, cols.pop(5))

In [533]:
df = df[cols]

In [534]:
df

,page,section,date,roundup,bulletin_url,final_url,bulletin_mps
0,https://web.archive.org/web/20210709115515/htt...,1.0,[9 July 2021],"[Infections have increased across the UK, Coro...",[{'text': 'Read the latest Coronavirus (COVID-...,https://www.ons.gov.uk/peoplepopulationandcomm...,"In England, the percentage of people testing p..."
1,https://web.archive.org/web/20210709115515/htt...,2.0,[9 July 2021],[Most report more social distancing than they ...,[{'text': 'Latest bulletin on Coronavirus and ...,https://www.ons.gov.uk/peoplepopulationandcomm...,"This week, over the period 30 June to 4 July 2..."
2,https://web.archive.org/web/20210709115515/htt...,3.0,[7 July 2021],[COVID-19 antibodies continue to rise in line ...,[{'text': 'Latest bulletin on antibody and vac...,https://www.ons.gov.uk/peoplepopulationandcomm...,"In England, it is estimated that around 9 in 1..."
3,https://web.archive.org/web/20210709115515/htt...,4.0,[6 July 2021],[Deaths in England and Wales fall below five-y...,[{'text': 'Latest bulletin on deaths registere...,https://www.ons.gov.uk/peoplepopulationandcomm...,"There were 8,690 deaths in England and Wales r..."
4,https://web.archive.org/web/20210709115515/htt...,5.0,[6 July 2021],[North West and London had highest coronavirus...,[{'text': 'Read Deaths registered due to COVID...,https://www.ons.gov.uk/peoplepopulationandcomm...,Of all deaths registered in 2020 in England an...
...,...,...,...,...,...,...,...
313,https://web.archive.org/web/20220426095424/htt...,1.0,[26 April 2022],[10.1% of registered deaths in England and Wal...,[{'text': 'Read the latest bulletin on deaths ...,https://www.ons.gov.uk/peoplepopulationandcomm...,"In the week ending 15 April 2022 (Week 15), 9,..."
314,https://web.archive.org/web/20220430123803/htt...,1.0,[29 April 2022],[Three out of four UK countries show a continu...,[{'text': 'Read the latest results from the Co...,https://www.ons.gov.uk/peoplepopulationandcomm...,In the week ending 23 April 2022:\nThe percent...
315,https://web.archive.org/web/20220430123803/htt...,2.0,[29 April 2022],[UK sickness absence rate hit 10-year high in ...,[{'text': 'Read the latest insights on COVID-1...,https://www.ons.gov.uk/employmentandlabourmark...,The sickness absence rate in the UK in 2021 ro...
316,https://web.archive.org/web/20220430123803/htt...,3.0,[28 April 2022],[54% rise in fraud and computer misuse offence...,[{'text': 'Read more in our Crime in England a...,https://www.ons.gov.uk/peoplepopulationandcomm...,Patterns of crime over the last two years have...


In [535]:
dfComb = df.append(dforig)

In [536]:
dfComb

,page,section,date,roundup,bulletin_url,final_url,bulletin_mps
0,https://web.archive.org/web/20210709115515/htt...,1.0,[9 July 2021],"[Infections have increased across the UK, Coro...",[{'text': 'Read the latest Coronavirus (COVID-...,https://www.ons.gov.uk/peoplepopulationandcomm...,"In England, the percentage of people testing p..."
1,https://web.archive.org/web/20210709115515/htt...,2.0,[9 July 2021],[Most report more social distancing than they ...,[{'text': 'Latest bulletin on Coronavirus and ...,https://www.ons.gov.uk/peoplepopulationandcomm...,"This week, over the period 30 June to 4 July 2..."
2,https://web.archive.org/web/20210709115515/htt...,3.0,[7 July 2021],[COVID-19 antibodies continue to rise in line ...,[{'text': 'Latest bulletin on antibody and vac...,https://www.ons.gov.uk/peoplepopulationandcomm...,"In England, it is estimated that around 9 in 1..."
3,https://web.archive.org/web/20210709115515/htt...,4.0,[6 July 2021],[Deaths in England and Wales fall below five-y...,[{'text': 'Latest bulletin on deaths registere...,https://www.ons.gov.uk/peoplepopulationandcomm...,"There were 8,690 deaths in England and Wales r..."
4,https://web.archive.org/web/20210709115515/htt...,5.0,[6 July 2021],[North West and London had highest coronavirus...,[{'text': 'Read Deaths registered due to COVID...,https://www.ons.gov.uk/peoplepopulationandcomm...,Of all deaths registered in 2020 in England an...
...,...,...,...,...,...,...,...
223,https://web.archive.org/web/20210808215813/htt...,3.0,['6 August 2021'],['A quarter of\u202fadults\u202fself-isolating...,[{'text': 'Read the latest bulletin on coronav...,https://www.ons.gov.uk/peoplepopulationandcomm...,"This week, over the period 28 July to 1 August..."
224,https://web.archive.org/web/20210808215813/htt...,4.0,['5 August 2021'],['1 in 68 people in the UK experiencing self-r...,[{'text': 'Read the latest bulletin on the pre...,http://ons.gov.uk/peoplepopulationandcommunity...,"An estimated 945,000 people living in private ..."
225,https://web.archive.org/web/20210808215813/htt...,5.0,['5 August 2021'],['Average UK daily flights up 117% since May 2...,[{'text': 'Read the latest bulletin on economi...,https://www.ons.gov.uk/economy/economicoutputa...,"In the week to 1 August 2021, the seven-day av..."
226,https://web.archive.org/web/20210808215813/htt...,6.0,['4 August 2021'],['9 out of 10 adults tested positive for COVID...,[{'text': 'Read more in the Coronavirus (COVID...,https://www.ons.gov.uk/peoplepopulationandcomm...,"In England, it is estimated that over 9 in 10 ..."


In [549]:
import ast

True

In [565]:
def procru(ru):
    if type(ru) == str:
        ru = ast.literal_eval(ru)
    stri = ''
    print(ru)
    for line in ru:
        stri = stri + line +'\n'
    return stri
    
procru(dfComb['roundup'][0])

0    [Infections have increased across the UK, Coro...
0    ['Number of COVID-19 infections continues to i...
Name: roundup, dtype: object


TypeError: can only concatenate str (not "list") to str

In [579]:
for i in dfComb.index:
    dfComb['roundupProc'][i] = procru(dfComb['roundup'][i])

['Infections have increased across the UK', 'Coronavirus (COVID-19) cases increased in England, Wales, Northern Ireland and Scotland in the week ending 3 July 2021.', 'The estimated percentage of the community population (those not in hospitals, care homes or other institutional settings) that had COVID-19 was:', ' - 0.61% in England (1 in 160 people)', ' - 0.30% in Wales (1 in 340 people)', ' - 0.33% in Northern Ireland (1 in 300 people)', ' - 1.01% in Scotland (1 in 100 people)', 'In England, infections were estimated to be highest in the North East and North West regions.', 'Across England overall, estimated infection rates increased in all age groups and were highest in those in school Year 12 (aged 16 and 17 years) to age 24 years.']
['Most report more social distancing than they perceive in others', "Most adults felt that compliance with social distancing measures and wearing a face-covering were “important” or “very important”. However, their perception of other people's complia

<ipython-input-579-3af2b32d79a2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfComb['roundupProc'][i] = procru(dfComb['roundup'][i])


In [562]:
dfComb

,page,section,date,roundup,bulletin_url,final_url,bulletin_mps,roundupProc
0,https://web.archive.org/web/20210709115515/htt...,1.0,[9 July 2021],"[Infections have increased across the UK, Coro...",[{'text': 'Read the latest Coronavirus (COVID-...,https://www.ons.gov.uk/peoplepopulationandcomm...,"In England, the percentage of people testing p...","[Infections have increased across the UK, Coro..."
1,https://web.archive.org/web/20210709115515/htt...,2.0,[9 July 2021],[Most report more social distancing than they ...,[{'text': 'Latest bulletin on Coronavirus and ...,https://www.ons.gov.uk/peoplepopulationandcomm...,"This week, over the period 30 June to 4 July 2...",[Most report more social distancing than they ...
2,https://web.archive.org/web/20210709115515/htt...,3.0,[7 July 2021],[COVID-19 antibodies continue to rise in line ...,[{'text': 'Latest bulletin on antibody and vac...,https://www.ons.gov.uk/peoplepopulationandcomm...,"In England, it is estimated that around 9 in 1...",[COVID-19 antibodies continue to rise in line ...
3,https://web.archive.org/web/20210709115515/htt...,4.0,[6 July 2021],[Deaths in England and Wales fall below five-y...,[{'text': 'Latest bulletin on deaths registere...,https://www.ons.gov.uk/peoplepopulationandcomm...,"There were 8,690 deaths in England and Wales r...",[Deaths in England and Wales fall below five-y...
4,https://web.archive.org/web/20210709115515/htt...,5.0,[6 July 2021],[North West and London had highest coronavirus...,[{'text': 'Read Deaths registered due to COVID...,https://www.ons.gov.uk/peoplepopulationandcomm...,Of all deaths registered in 2020 in England an...,[North West and London had highest coronavirus...
...,...,...,...,...,...,...,...,...
223,https://web.archive.org/web/20210808215813/htt...,3.0,['6 August 2021'],['A quarter of\u202fadults\u202fself-isolating...,[{'text': 'Read the latest bulletin on coronav...,https://www.ons.gov.uk/peoplepopulationandcomm...,"This week, over the period 28 July to 1 August...",['A quarter of\u202fadults\u202fself-isolating...
224,https://web.archive.org/web/20210808215813/htt...,4.0,['5 August 2021'],['1 in 68 people in the UK experiencing self-r...,[{'text': 'Read the latest bulletin on the pre...,http://ons.gov.uk/peoplepopulationandcommunity...,"An estimated 945,000 people living in private ...",['1 in 68 people in the UK experiencing self-r...
225,https://web.archive.org/web/20210808215813/htt...,5.0,['5 August 2021'],['Average UK daily flights up 117% since May 2...,[{'text': 'Read the latest bulletin on economi...,https://www.ons.gov.uk/economy/economicoutputa...,"In the week to 1 August 2021, the seven-day av...",['Average UK daily flights up 117% since May 2...
226,https://web.archive.org/web/20210808215813/htt...,6.0,['4 August 2021'],['9 out of 10 adults tested positive for COVID...,[{'text': 'Read more in the Coronavirus (COVID...,https://www.ons.gov.uk/peoplepopulationandcomm...,"In England, it is estimated that over 9 in 10 ...",['9 out of 10 adults tested positive for COVID...


In [570]:
dfComb['page'][0]

0    https://web.archive.org/web/20210709115515/htt...
0    https://web.archive.org/web/20201109120409/htt...
Name: page, dtype: object

In [569]:
[i for i in dfComb['roundup'][0]]

[['Infections have increased across the UK',
  'Coronavirus (COVID-19) cases increased in England, Wales, Northern Ireland and Scotland in the week ending 3 July 2021.',
  'The estimated percentage of the community population (those not in hospitals, care homes or other institutional settings) that had COVID-19 was:',
  ' - 0.61% in England (1 in 160 people)',
  ' - 0.30% in Wales (1 in 340 people)',
  ' - 0.33% in Northern Ireland (1 in 300 people)',
  ' - 1.01% in Scotland (1 in 100 people)',
  'In England, infections were estimated to be highest in the North East and North West regions.',
  'Across England overall, estimated infection rates increased in all age groups and were highest in those in school Year 12 (aged 16 and 17 years) to age 24 years.'],
 "['Number of COVID-19 infections continues to increase across UK', 'The number of people with the coronavirus (COVID-19) in England and Wales continues to increase but the rate of increase is slower than it has been in recent weeks.

In [571]:
dfCombCopy = dfComb.copy()

In [576]:
dfComb=dfComb.reset_index()

In [577]:
dfComb=dfComb.drop(columns=['index'])

In [580]:
dfComb

,page,section,date,roundup,bulletin_url,final_url,bulletin_mps,roundupProc
0,https://web.archive.org/web/20210709115515/htt...,1.0,[9 July 2021],"[Infections have increased across the UK, Coro...",[{'text': 'Read the latest Coronavirus (COVID-...,https://www.ons.gov.uk/peoplepopulationandcomm...,"In England, the percentage of people testing p...",Infections have increased across the UK\nCoron...
1,https://web.archive.org/web/20210709115515/htt...,2.0,[9 July 2021],[Most report more social distancing than they ...,[{'text': 'Latest bulletin on Coronavirus and ...,https://www.ons.gov.uk/peoplepopulationandcomm...,"This week, over the period 30 June to 4 July 2...",Most report more social distancing than they p...
2,https://web.archive.org/web/20210709115515/htt...,3.0,[7 July 2021],[COVID-19 antibodies continue to rise in line ...,[{'text': 'Latest bulletin on antibody and vac...,https://www.ons.gov.uk/peoplepopulationandcomm...,"In England, it is estimated that around 9 in 1...",COVID-19 antibodies continue to rise in line w...
3,https://web.archive.org/web/20210709115515/htt...,4.0,[6 July 2021],[Deaths in England and Wales fall below five-y...,[{'text': 'Latest bulletin on deaths registere...,https://www.ons.gov.uk/peoplepopulationandcomm...,"There were 8,690 deaths in England and Wales r...",Deaths in England and Wales fall below five-ye...
4,https://web.archive.org/web/20210709115515/htt...,5.0,[6 July 2021],[North West and London had highest coronavirus...,[{'text': 'Read Deaths registered due to COVID...,https://www.ons.gov.uk/peoplepopulationandcomm...,Of all deaths registered in 2020 in England an...,North West and London had highest coronavirus ...
...,...,...,...,...,...,...,...,...
500,https://web.archive.org/web/20210808215813/htt...,3.0,['6 August 2021'],['A quarter of\u202fadults\u202fself-isolating...,[{'text': 'Read the latest bulletin on coronav...,https://www.ons.gov.uk/peoplepopulationandcomm...,"This week, over the period 28 July to 1 August...",A quarter of adults self-isolating were notifi...
501,https://web.archive.org/web/20210808215813/htt...,4.0,['5 August 2021'],['1 in 68 people in the UK experiencing self-r...,[{'text': 'Read the latest bulletin on the pre...,http://ons.gov.uk/peoplepopulationandcommunity...,"An estimated 945,000 people living in private ...",1 in 68 people in the UK experiencing self-rep...
502,https://web.archive.org/web/20210808215813/htt...,5.0,['5 August 2021'],['Average UK daily flights up 117% since May 2...,[{'text': 'Read the latest bulletin on economi...,https://www.ons.gov.uk/economy/economicoutputa...,"In the week to 1 August 2021, the seven-day av...",Average UK daily flights up 117% since May 202...
503,https://web.archive.org/web/20210808215813/htt...,6.0,['4 August 2021'],['9 out of 10 adults tested positive for COVID...,[{'text': 'Read more in the Coronavirus (COVID...,https://www.ons.gov.uk/peoplepopulationandcomm...,"In England, it is estimated that over 9 in 10 ...",9 out of 10 adults tested positive for COVID-1...


In [582]:
dfComb['len_ratio'] = 'ph'

In [584]:
dfComb['rougef1'] = 'ph'

In [585]:
from rouge import Rouge 
rouge = Rouge()

In [587]:
for i in dfComb.index:
    dfComb['rougef1'][i] = rouge.get_scores(dfComb['roundupProc'][i], dfComb['bulletin_mps'][i])[0]['rouge-2']['f']

<ipython-input-587-3c1773f7e809>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfComb['rougef1'][i] = rouge.get_scores(dfComb['roundupProc'][i], dfComb['bulletin_mps'][i])[0]['rouge-2']['f']


In [588]:
for i in dfComb.index:
    dfComb['len_ratio'][i] = len(dfComb['bulletin_mps'][i]) / len(dfComb['roundupProc'][i])

<ipython-input-588-1a62b5aa3620>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfComb['len_ratio'][i] = len(dfComb['bulletin_mps'][i]) / len(dfComb['roundupProc'][i])


In [589]:
dfCombLowRouge = dfComb[dfComb['rougef1'] < 0.15]
dfComb = dfComb[dfComb['rougef1'] > 0.15]

In [590]:
dfCombLowLen = dfComb[dfComb['len_ratio'] < 1]
dfComb = dfComb[dfComb['len_ratio'] > 1]

In [635]:
dfComb

,page,section,date,roundup,bulletin_url,final_url,bulletin_mps,roundupProc,len_ratio,rougef1
0,https://web.archive.org/web/20210709115515/htt...,1.0,[9 July 2021],"[Infections have increased across the UK, Coro...",[{'text': 'Read the latest Coronavirus (COVID-...,https://www.ons.gov.uk/peoplepopulationandcomm...,"In England, the percentage of people testing p...",Infections have increased across the UK\nCoron...,1.77254,0.192469
1,https://web.archive.org/web/20210709115515/htt...,2.0,[9 July 2021],[Most report more social distancing than they ...,[{'text': 'Latest bulletin on Coronavirus and ...,https://www.ons.gov.uk/peoplepopulationandcomm...,"This week, over the period 30 June to 4 July 2...",Most report more social distancing than they p...,2.43585,0.183486
5,https://web.archive.org/web/20210709115515/htt...,6.0,[2 July 2021],[Almost 9 in 10 adults say they have had their...,[{'text': 'Read the latest bulletin on coronav...,https://www.ons.gov.uk/peoplepopulationandcomm...,"In the latest period, 26 May to 20 June 2021, ...",Almost 9 in 10 adults say they have had their ...,1.49144,0.196382
6,https://web.archive.org/web/20210713150217/htt...,1.0,[13 July 2021],[Deaths from COVID-19 increased slightly in En...,[{'text': 'Read the latest bulletin on coronav...,https://www.ons.gov.uk/peoplepopulationandcomm...,"There were 8,808 deaths in England and Wales r...",Deaths from COVID-19 increased slightly in Eng...,1.10044,0.18
8,https://web.archive.org/web/20210715104624/htt...,1.0,[15 July 2021],[Just over 4 in 10 businesses provided no trai...,[{'text': 'Read the latest bulletin on busines...,https://www.ons.gov.uk/businessindustryandtrad...,The proportion of businesses' workforce report...,Just over 4 in 10 businesses provided no train...,1.19905,0.162319
...,...,...,...,...,...,...,...,...,...,...
499,https://web.archive.org/web/20210808215813/htt...,2.0,['6 August 2021'],"['Infections decrease in England, Wales and Sc...",[{'text': 'Read the latest Coronavirus (COVID-...,https://www.ons.gov.uk/peoplepopulationandcomm...,"In England, the percentage of people testing p...","Infections decrease in England, Wales and Scot...",1.01665,0.178723
500,https://web.archive.org/web/20210808215813/htt...,3.0,['6 August 2021'],['A quarter of\u202fadults\u202fself-isolating...,[{'text': 'Read the latest bulletin on coronav...,https://www.ons.gov.uk/peoplepopulationandcomm...,"This week, over the period 28 July to 1 August...",A quarter of adults self-isolating were notifi...,2.32704,0.22906
501,https://web.archive.org/web/20210808215813/htt...,4.0,['5 August 2021'],['1 in 68 people in the UK experiencing self-r...,[{'text': 'Read the latest bulletin on the pre...,http://ons.gov.uk/peoplepopulationandcommunity...,"An estimated 945,000 people living in private ...",1 in 68 people in the UK experiencing self-rep...,1.85681,0.504348
503,https://web.archive.org/web/20210808215813/htt...,6.0,['4 August 2021'],['9 out of 10 adults tested positive for COVID...,[{'text': 'Read more in the Coronavirus (COVID...,https://www.ons.gov.uk/peoplepopulationandcomm...,"In England, it is estimated that over 9 in 10 ...",9 out of 10 adults tested positive for COVID-1...,2.13361,0.412017


In [592]:
df_high_rouge_lenE = pd.read_csv('./econ_high_rouge_len.csv')

In [618]:
len(df_high_rouge_lenE)

53

In [597]:
import random

In [631]:
roundupsum_train = {}
roundupsum_train['data'] = []

roundupsum_val = {}
roundupsum_val['data'] = []

In [636]:
for i in dfComb.index:
    if random.random() < 0.05:
        roundupsum_val['data'].append({
            'document': dfComb.loc[i]['bulletin_mps'],
            'summary': dfComb.loc[i]['roundupProc'],
            'id': i
        })
    else:
        roundupsum_train['data'].append({
            'document': dfComb.loc[i]['bulletin_mps'],
            'summary': dfComb.loc[i]['roundupProc'],
            'id': i
        })

In [637]:
len(roundupsum_train['data'])

334

In [626]:
with open('roundupsum_train.json', 'w') as outfile:
    json.dump(roundupsum_train, outfile)
    
with open('roundupsum_val.json', 'w') as outfile:
    json.dump(roundupsum_val, outfile)

In [627]:
! git add .
! git commit -m "first commit"
! git push -u origin main

[main 42508f5] first commit
 9 files changed, 9661 insertions(+), 281 deletions(-)
 create mode 100644 .ipynb_checkpoints/CovSelenium-Copy1-checkpoint.ipynb
 create mode 100644 CovSelenium-Copy1.ipynb
 create mode 100644 covidrounduporig.csv
 rewrite roundupsum_train.json (85%)
 rewrite roundupsum_val.json (99%)
Enumerating objects: 19, done.
Counting objects: 100% (19/19), done.
Delta compression using up to 16 threads
Compressing objects: 100% (12/12), done.
Writing objects: 100% (12/12), 372.28 KiB | 4.14 MiB/s, done.
Total 12 (delta 9), reused 0 (delta 0)
remote: Resolving deltas: 100% (9/9), completed with 5 local objects.
To https://github.com/theojolliffe/roundupdata.git
   491b92c..42508f5  main -> main
Branch 'main' set up to track remote branch 'main' from 'origin'.
